stop phrases
'join us'
'days left'
'mark your calendars'

In [ ]:
import tweepy
#import pandas as pd
from collections import defaultdict
import re
import json
import os
from pprint import pprint
from googletrans import Translator, constants
import time
from datetime import datetime

import jmespath

In [ ]:

date = datetime.today().strftime('%Y-%m-%d')
weekly_filename = f'deduped_weekly_tweets_{date}.json'
URL_DUPES={}

In [ ]:
def loadSavedMappings():
    
    AUTHOR_MAP = {}
    
    #author mappings
    map_file = 'authors.json'
    
    fullFilePath = f'twitter_data/saved_mappings/{map_file}'
    with open(fullFilePath, 'r') as f:
        AUTHOR_MAP = json.load(f)
    
    f.close()
    
    return AUTHOR_MAP

AUTHOR_MAP = loadSavedMappings()


In [ ]:
def get_local_author(author_id):
    
    
    auth_list = jmespath.search(f"data[?id == '{author_id}'].username", AUTHOR_MAP)


    if len(auth_list) == 1:
        author_username = auth_list[0]
    elif len(auth_list) > 1:
        author_username = 'UNKN_MultipleResultsReturned'
    else:
        author_username = author_id


    return author_username


#test
author_username = get_local_author('774273192395280384')

In [ ]:
def scrubText(tweetText):
    
    #remove newlines
    tweetText = re.sub('\n', ' ', tweetText)
    
    #remove amp
    tweetText = re.sub('&amp;', ' and ', tweetText)
    
      #remove pipes because I use them as delimiters
    tweetText = tweetText.replace('|', ' ')  
    
    #remove unicode special chars
    string_encode = tweetText.encode("ascii", "ignore")
    tweetText = string_encode.decode()
    
    return tweetText

scrubText('asdfa|sdf &amp; dsdfgsdfg')

In [ ]:
#Initiating Varibales -- derived programatically or rarely changed

userName = 'defaultUserName'

#MAPPING_FILE_DIR = 'twitter_data/saved_mappings/'
weekly_file = f'twitter_data/weekly_files/{weekly_filename}'
baseTweetURL = 'https://twitter.com/{userName}/status/{tweetId}'


In [ ]:
def remove_twitter_urls(urls):
    
    #capture urls
    patterns = [re.compile("pic\.twitter\.com"), 
                re.compile("twitter\.com")]
    returnUrl = []
    
    for url in urls:
        include = 'Y'
        for pattern in patterns:
            if pattern.search(url) != None:
                include = 'N'
        if include == 'Y':
            returnUrl.append(url)
    
    return returnUrl

In [ ]:
def check_for_url_dupes(url):
    dupe = 0
    if url in URL_DUPES:
        URL_DUPES[url] += 1
        dupe = 1
        
    else:
        URL_DUPES[url] = 0
        
    return dupe
    

In [ ]:
URL_DUPES = {}
iterr = 0
rollingContent = ''
summaryDF = []

weekly_file = f'twitter_data/weekly_files/{weekly_filename}'

summaryFilePath = f'twitter_data/weekly_summaries/summary_{date}.txt'
with open(summaryFilePath, 'w') as summaryFile:

    
    

    summaryFile.write('reportDate|s_tweetURL|s_referenceURL|tweetId|author|tweetURL|referenceURL|tweetText|PromoteIndicator|Time Sensitive Flag|Use Case|Technical Underpinning|Verbiage')
    summaryFile.write("\n")
    
    summaryHeaderDF = ['tweetId','author','likes','retweets','tweetURL','tweetText','referenceUrls']
    
    
    with open(weekly_file, 'r') as f:
        tweets = json.load(f)


        for tweet in tweets['data']:


            iterr +=1



            #print(tweet)
            author = get_local_author(jmespath.search("author_id", tweet))
            text = jmespath.search("text", tweet)
            text = scrubText(text)
            likes = jmespath.search("public_metrics.like_count", tweet)
            retweets = jmespath.search("public_metrics.retweet_count", tweet)
            urls = jmespath.search("entities.urls[*].expanded_url", tweet)
            urls = remove_twitter_urls(urls)

      
            url_dupe_flag=0
            if len(urls) == 0:
                url_dupe_flag = 1 #dupe or no url is functionally the same
            else:    
                url_dupe_flag = check_for_url_dupes(urls[0])

            if url_dupe_flag == 1:
                continue
            
            stringUrl =''
            firstURL = urls[0]
            for url in urls:
                stringUrl += '|' + url

            tweetId = jmespath.search("id", tweet)

            tweetURL = f'https://twitter.com/{author}/status/{tweetId}'

            blank=''
            content = date + "|" + tweetURL+ "|"+ firstURL + "|" + str(tweetId)+ "|"+author+ "|||"+text
            contentDF = [tweetId,author,tweetURL,text,urls]
            summaryDF.append(contentDF)
            rollingContent = rollingContent + ' ' + text
            summaryFile.write(content)
            summaryFile.write("\n")
            #time.sleep(2)
    f.close()
summaryFile.close()
                


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer



#nltk.download('stopwords')
stop_words = stopwords.words('english')

new_stopwords = ['https']

stop_words.extend(new_stopwords)

word_data = rollingContent
word_data = word_data.lower()
nltk_tokens = nltk.word_tokenize(word_data)
nltk_tokens= [word for word in nltk_tokens if word.isalnum()]



filtered_sentence = [w for w in nltk_tokens if not w.lower() in stop_words]
 
filtered_sentence = []
 
for w in nltk_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)
 

print(filtered_sentence)

print(list(nltk.bigrams(nltk_tokens)))

In [ ]:
bigrams = nltk.bigrams(filtered_sentence)
fdist = nltk.FreqDist(bigrams)

fdist.most_common(100)

#for k,v in fdist.items():
#    print (k,v)


In [ ]:
fdist

In [ ]:
#compute frequency distribution for all the bigrams in the text
fdist = nltk.FreqDist(bigrams)


In [ ]:
x = []
y = ['s','d']
z = ['a','b']

x.append(y)
x.append(z)

print(x)

In [ ]:
import pandas as pd

df = pd.DataFrame (summaryDF, columns = summaryHeaderDF)

In [ ]:
df['tweetText']

In [ ]:
regex = '.*diamcircle.*'

df['lc_tweetText'] = df['tweetText'].str.lower()

df[df.lc_tweetText.str.match(regex)]

In [ ]:
len(URL_DUPES)